In [2]:
import sqlite3
import psycopg2
import pandas as pd
from DataBase_functions import *
from DataBase_functions_testing import *

#Google sheed API
import gspread
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

<p style="font-size: 30px; text-align: center; color: #20a7e5;">Creating data-frame</p>

<p style="font-size: 25px; color: #208ee5">Registro actividad todos</p>

In [3]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "12qLRAp7C9qzcoWkyUTJAVMhRHMFfwYec094J8CU2t7c" #Registro actividad todos: 12qLRAp7C9qzcoWkyUTJAVMhRHMFfwYec094J8CU2t7c

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #Dataframe con los datos de todos los meses
    df = pd.DataFrame(columns=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]) 
    meses = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
    for sheet in sheets:
        sheet_name = sheet.title 
        if sheet_name in meses:
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]))
            df = pd.concat([df, df_aux], ignore_index=True)
        else: 
            print(sheet_name, "is not a month")
            pass
    #df = df.drop_duplicates(subset='Código', keep='first')
    print(df.shape)

    #Dataframe with 2024 sheet data
    sheet_2024 = spreadsheet.worksheet("2024")
    df_2024 = pd.DataFrame(sheet_2024.get_all_records(expected_headers=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]))
    #df_2024 = df_2024.drop_duplicates(subset='Código', keep='first')
    print(df_2024.shape)

    df = pd.concat([df, df_2024], ignore_index=True)
    df = df.drop_duplicates(subset="Código", keep="first")
    print(df.shape)

    #Dataframe validation
    df.rename(columns={"Validada": "validada", "Responsable": "responsable", "Gestor DDBB": "DDBB", "Geografía": "geography", "Petición": "petition_code", "Feature": "feature", "SDATOOL": "sdatool", "Código": "DQDP_code", "Gestor BBDD": "DDBB", "Horas": "duration_time", "Comentarios": "description", "Fecha de alta": "fecha_in", "Fecha de fin": "fecha_out"}, inplace=True)

    df["fecha_in"] = df["fecha_in"].astype(str)
    df["Fecha incurrida"] = df["Fecha incurrida"].astype(str)
    df["fecha_out"] = df["fecha_out"].astype(str)
    df["UUAA"] = df["UUAA"].astype(str)
    df["DQDP_code"] = df["DQDP_code"].astype(str)
    df["sdatool"] = df["sdatool"].astype(str)
    df["feature"] = df["feature"].astype(str)
    df["geography"] = df["geography"].astype(str)
    df["DDBB"] = df["DDBB"].astype(str)
    df["Ámbito"] = df["Ámbito"].astype(str)
    df["responsable"] = df["responsable"].astype(str)
    df["validada"] = df["validada"].astype(str)
    df["duration_time"] = pd.to_numeric(df["duration_time"], errors="coerce")
    df["description"] = df["description"].astype(str)
    df["petition_code"] = df["petition_code"].astype(str)

    df["version_date"] = "Nan"
    df["version"] = "Nan"
    df["petition_arq"] = "Nan"
    df["dev_master"] = "Nan"
    df["estado"] = "Nan"

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['2024 totales - Peticiones', '2024 totales - Peticiones - Pro', '2024', 'Gráficos Semanales - Procesos', '2024 - Procesos', 'Abril', 'Mayo', 'Julio', 'Junio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre', 'Responsables']
2024 totales - Peticiones is not a month
2024 totales - Peticiones - Pro is not a month
2024 is not a month
Gráficos Semanales - Procesos is not a month
2024 - Procesos is not a month
Responsables is not a month
(4371, 18)
(1974, 18)
(1739, 18)


In [4]:
for row in df.iterrows(): 
    for key, value in dict(row[1]).items():
        if value == "Nan" or value == None or value == "" or value == "None": 
            if key == "petition_code" or key == "DQDP_code" or key == "sdatool" or key == "feature" or key == "UUAA" or key == "version" or key == "petition_arq" or key == "description":
                df.at[row[0], key] = "None"
            else: 
                df.at[row[0], key] = None
print(df.shape, df.columns)
print(df.isna().sum())

(1739, 23) Index(['fecha_in', 'Fecha incurrida', 'fecha_out', 'UUAA', 'DQDP_code',
       'sdatool', 'feature', 'petition_code', 'geography', 'DDBB', 'Ámbito',
       'responsable', 'validada', 'duration_time', 'description', 'Estado',
       'Días', '', 'version_date', 'version', 'petition_arq', 'dev_master',
       'estado'],
      dtype='object')
fecha_in            263
Fecha incurrida      55
fecha_out           391
UUAA                  0
DQDP_code             0
sdatool               0
feature               0
petition_code         0
geography           268
DDBB                170
Ámbito              170
responsable         631
validada            854
duration_time        72
description           0
Estado             1150
Días               1296
                    458
version_date       1739
version               0
petition_arq          0
dev_master         1739
estado             1739
dtype: int64


<p style="font-size: 25px; color: #208ee5; text-align: center;">Peticiones globales Oracle, Elastic Search y Mongo DB</p>

<p style="font-size: 25px; color: #208cc5;;">Oracle Physics</p>

In [5]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1PZKCpybEechpbwP-H3khhUihHqwUqgbi_KkM6iCxhuE" #Peticiones globales Oracle: "1PZKCpybEechpbwP-H3khhUihHqwUqgbi_KkM6iCxhuE"

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    df_OG = pd.DataFrame(columns=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"])

    for sheet in sheets: 
        try: 
            sheet_name = sheet.title
            
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"]))
            df_OG = pd.concat([df_OG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_OG["DDBB"] = "Oracle Physics"
    df_OG["geography"] = "Global"

    print(df_OG.shape)

    df_OG["UUAA"] = df_OG["UUAA"].astype(str)
    df_OG["Petición Jira"] = df_OG["Petición Jira"].astype(str)
    df_OG["Versión Repo"] = df_OG["Versión Repo"].astype(str)
    df_OG["Petición ARQ"] = df_OG["Petición ARQ"].astype(str)
    df_OG["Fecha cierre"] = df_OG["Fecha cierre"].astype(str)
    df_OG["Comentarios"] = df_OG["Comentarios"].astype(str)
    df_OG["Path"] = df_OG["Path"].astype(str)
    df_OG["DDBB"] = df_OG["DDBB"].astype(str)
    df_OG["geography"] = df_OG["geography"].astype(str)

    df_OG.rename(columns={"Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['APIC', 'ATAU', 'B1DS', 'B43T', 'BJP9', 'CCUS', 'DAMS', 'DMTO', 'EGRP', 'EI4M', 'ESCD', 'FZHI', 'GDEL', 'GHDC', 'GJSX', 'GSIN', 'GUAR', 'H9IY', 'HFPP', 'HLCY', 'HPGR', 'IMPT', 'J6G7', 'K5VI', 'KAPI', 'KARC', 'KBGE', 'KBTQ', 'KCCX', 'KCDO', 'KCMC', 'KCMS', 'KCNC', 'KCNS', 'KCOG', 'KCOL', 'KCSN', 'KCSP', 'KDCL', 'KDFH', 'KDMG', 'KERF', 'KESJ', 'KFGC', 'KFRE', 'KFUL', 'KGCC', 'KGCI', 'KGTP', 'KIFV', 'KIVS', 'KLIM', 'KLNE', 'KMET', 'KMIC', 'KMOL', 'KMRK', 'KMYC', 'KNWD', 'KOMN', 'KONB', 'KPAG', 'KPAD', 'KPAY', 'KPDA', 'KPDR', 'KPFE', 'KPFM', 'KPIG', 'KPPS', 'KPST', 'KRBA', 'KRCE', 'KSAG', 'KSAN', 'KSDD', 'KSPH', 'KSRO', 'KUSU', 'KVCS', 'KXSG', 'KYSZ', 'KYOP', 'KYUB', 'MCRR', 'MFCG ', 'MK0W', 'MDDM', 'NI0C', 'O43T', 'PGVC', 'PIVU', 'PMSC', 'PMSV', 'P6ES', 'QWJR', 'QWPJ', 'RLR7', 'SWP3', 'S47B', 'UGX2', 'V34F', 'VAMO', 'W1BD', 'XBOB', 'ZGR7', 'XMBR', 'XR6Z', 'ZDNM', 'ZNXD']
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
(945, 10)


<p style="font-size: 25px; color: #208cc5;;">Elastic Search</p>

In [14]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1PXFKrrJVs12XpmgV5DmnBvjxENQ3hobwLs-bdDf8OIE" #Peticiones globales Mongo: "1PXFKrrJVs12XpmgV5DmnBvjxENQ3hobwLs-bdDf8OIE"

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    df_EG = pd.DataFrame(columns=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"])

    for sheet in sheets: 
        try: 
            sheet_name = sheet.title
            
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"]))
            df_EG = pd.concat([df_EG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_EG["DDBB"] = "Oracle Physics"
    df_EG["geography"] = "Global"

    #df = df_OG.drop_duplicates(subset='Código', keep='first')
    print(df_EG.shape)

    df_EG["UUAA"] = df_EG["UUAA"].astype(str)
    df_EG["Petición Jira"] = df_EG["Petición Jira"].astype(str)
    df_EG["Versión Repo"] = df_EG["Versión Repo"].astype(str)
    df_EG["Petición ARQ"] = df_EG["Petición ARQ"].astype(str)
    df_EG["Fecha cierre"] = df_EG["Fecha cierre"].astype(str)
    df_EG["Comentarios"] = df_EG["Comentarios"].astype(str)
    df_EG["Path"] = df_EG["Path"].astype(str)
    df_EG["DDBB"] = df_EG["DDBB"].astype(str)
    df_EG["geography"] = df_EG["geography"].astype(str)

    df_EG.rename(columns={"Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['ATAU', 'F30V', 'EI4M', 'GHDC', 'GUAR', 'H9IY', 'IMPT', 'J6G7', 'JBSY', 'KAGE', 'KAPI', 'KBTQ', 'KCNS', 'KHR1', 'KIFV', 'KLNE', 'KMOL', 'KREC', 'KTIN', 'KTRA', 'KTR2', 'KXSG', 'KYGU', 'KYOP', 'L1WI', 'NI0C', 'O43T', 'OPEI', 'PGVC', 'S47B', 'XBMR', 'XPI2', 'YM3X', 'ZNXD']
(262, 10)


<p style="font-size: 25px; color: #208cc5;">Mongo DB</p>

In [15]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1-mO_2AFa0J4VBlONUFPuQ7IfTRNxBb3BbPyEg_D-FPs" #Peticiones globales Mongo: "1-mO_2AFa0J4VBlONUFPuQ7IfTRNxBb3BbPyEg_D-FPs"

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    df_MG = pd.DataFrame(columns=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"])

    for sheet in sheets: 
        try: 
            sheet_name = sheet.title
            
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"]))
            df_MG = pd.concat([df_MG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_MG["DDBB"] = "Oracle Physics"
    df_MG["geography"] = "Global"
    
    print(df_MG.shape)

    df_MG["UUAA"] = df_MG["UUAA"].astype(str)
    df_MG["Petición Jira"] = df_MG["Petición Jira"].astype(str)
    df_MG["Versión Repo"] = df_MG["Versión Repo"].astype(str)
    df_MG["Petición ARQ"] = df_MG["Petición ARQ"].astype(str)
    df_MG["Fecha cierre"] = df_MG["Fecha cierre"].astype(str)
    df_MG["Comentarios"] = df_MG["Comentarios"].astype(str)
    df_MG["Path"] = df_MG["Path"].astype(str)
    df_MG["DDBB"] = df_MG["DDBB"].astype(str)
    df_MG["geography"] = df_MG["geography"].astype(str)

    df_MG.rename(columns={"Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)
    
except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['KSKR', 'KPDR', 'F30V', 'KAPI', 'H9IY', 'KPAD', 'CHM8', 'XMBR', 'KCOG', 'KONL', 'KNWD', 'KIFV', 'KIFD', 'KPFM', 'KHR1', 'KARC', 'KATT']
(77, 9)


<p style="font-size: 25px; color: #20cff5;">All in one Data Frame</p>

In [16]:
print(df.columns, df_OG.columns, df_EG.columns, df_MG.columns)

Index(['fecha_in', 'Fecha incurrida', 'fecha_out', 'UUAA', 'DQDP_code',
       'sdatool', 'feature', 'petition_code', 'geography', 'DDBB', 'Ámbito',
       'responsable', 'validada', 'duration_time', 'description', 'Estado',
       'Días', '', 'version_date', 'version', 'petition_arq', 'dev_master',
       'estado', 'Path', 'Petición Jira', 'Versión Repo', 'Petición ARQ',
       'Fecha cierre', 'Comentarios'],
      dtype='object') Index(['UUAA', 'petition_code', 'version', 'petition_arq', 'fecha_out',
       'description', 'Path', '', 'DDBB', 'geography'],
      dtype='object') Index(['UUAA', 'petition_code', 'version', 'petition_arq', 'fecha_out',
       'description', 'Path', '', 'DDBB', 'geography'],
      dtype='object') Index(['UUAA', 'petition_code', 'version', 'petition_arq', 'fecha_out',
       'description', 'Path', 'DDBB', 'geography'],
      dtype='object')


In [17]:
df_aux = pd.DataFrame(columns=list(df.columns))
df_aux = pd.concat([df_aux, df_OG, df_EG, df_MG], ignore_index=True)
df_aux.drop_duplicates(subset=["petition_code", "version"], keep="first", inplace=True)
df = pd.concat([df, df_aux], ignore_index=True)

df["fecha_in"] = df["fecha_in"].astype(str)
df["Fecha incurrida"] = df["Fecha incurrida"].astype(str)
df["fecha_out"] = df["fecha_out"].astype(str)
df["UUAA"] = df["UUAA"].astype(str)
df["DQDP_code"] = df["DQDP_code"].astype(str)
df["sdatool"] = df["sdatool"].astype(str)
df["feature"] = df["feature"].astype(str)
df["geography"] = df["geography"].astype(str)
df["DDBB"] = df["DDBB"].astype(str)
df["Ámbito"] = df["Ámbito"].astype(str)
df["responsable"] = df["responsable"].astype(str)
df["validada"] = df["validada"].astype(str)
df["duration_time"] = pd.to_numeric(df["duration_time"], errors="coerce")
df["description"] = df["description"].astype(str)
df["version_date"] = df["version_date"].astype(str)
df["version"] = df["version"].astype(str)
df["petition_arq"] = df["petition_arq"].astype(str)
df["estado"] = df["estado"].astype(str)
df["petition_code"] = df["petition_code"].astype(str)

df["dev_master"] = "None"

/var/folders/r3/_0ct4tcx0jj4q9jfbhwjb93m0000gn/T/ipykernel_1609/3202183205.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_aux], ignore_index=True)


In [69]:
print(df[df["DQDP_code"].duplicated(keep="last") == True]["DQDP_code"].shape)
print(df[df["DQDP_code"] == "nan"].shape)

(2106,)
(2107, 29)


---
---

In [11]:
'''
for row in df.iterrows():
    insert_data_testing(dict(row[1]))
'''

'\nfor row in df.iterrows():\n    insert_data_testing(dict(row[1]))\n'

<p style="font-size: 40px; color: grey;">Testing</p>

In [12]:
diccionario = {"Nombre": "Matias", "Apellido": "Blaquier", "Edad": 27}
for key, value in diccionario.items(): 
    print(value)

###########
str_test = " "
print(len(str_test))
print(len(str_test.strip()))

Matias
Blaquier
27
1
0
